# item 1 - Analyze the dataset
Parse the groundtruth information into a dictionary and a pandas table

In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

%matplotlib inline

import numpy as np
import os, sys
import imageio
import pandas as pd
import matplotlib.pyplot as plt

path_txt = os.path.join('dataset', 'train', 'gt')
path_mask = os.path.join('dataset', 'train', 'mask')

dirs_txt = os.listdir(path_txt)
dirs_mask = os.listdir(path_mask)

data = dict()

for gt in dirs_txt:
    with open(os.path.join(path_txt, gt)) as f:
        lines = f.readlines()
        
        im_name = gt.replace('gt', 'mask').replace('txt', 'png')
        im_open = (imageio.imread(os.path.join(path_mask, im_name)))

        lista = list()
        
        # Extract grountruth information
        for l in lines:
            tly, tlx, bry, brx, tipo = l.split(' ')  
            tly, tlx, bry, brx = map(float, [tly, tlx, bry, brx])
            d = dict()
            d['type'] = tipo.strip()
            
            w = brx - tlx
            h = bry - tly
            
            d['width'] = w
            d['height'] = h
            d['bbox_area'] = w*h
            d['form_factor'] = w/h
            
            d['tly'] = round(tly)
            d['tlx'] = round(tlx)
            d['bry'] = round(bry)
            d['brx'] = round(brx)
            
            sub_mask = im_open[d['tly']:d['bry'], d['tlx']:d['brx']]
            mask_area = np.count_nonzero(sub_mask)
            d['mask_area'] = mask_area
            d['filling_ratio'] = mask_area / d['bbox_area']
            
            lista.append(d)
            
        data[gt] = lista
    

In [ ]:
# Show information in pandas format
columns = ['type','width','height','form_factor','bbox_area','mask_area','filling_ratio']
df = pd.DataFrame.from_dict({(i,n): data[i][n]
                        for i in data.keys()
                        for n,v in enumerate(data[i])}, columns=columns, orient='index').sort_values(['type'])
df

In [ ]:
print("Maximum values")
type_counts = df.groupby('type').aggregate(max)
print(type_counts)

type_counts = df.groupby('type').aggregate(min)
print("\n\nMinimum values")
print(type_counts)

print("\n\nSummatory")
type_counts = df.drop(columns=['form_factor','filling_ratio']).groupby('type').aggregate(sum)
print(type_counts)

type_counts['mask_area'].plot(figsize=(12, 8),kind='pie',sort_columns=True, title="Number of pixels per class")
normalized_df=type_counts['mask_area']/type_counts['mask_area'].sum()
print("\n\nPercentage of pixels per class")
print(normalized_df)


### CONCLUSIONS: 
Large range of sizes: 
- From 90 pixels (about 30x30) to 55919 of area


Total number of pixels of each class is very different:
- class F: 42%
- class B: 2%

In [ ]:
type_counts = df.groupby('type').aggregate(np.average).round(2)
print(type_counts)
type_counts.drop(columns=['width','height','form_factor']).plot(figsize=(5, 8),kind='bar',sort_columns=True,subplots=True)

type_counts = df.groupby('type').aggregate(np.std).round(3)
print(type_counts)

type_counts = df.groupby('type').aggregate(np.median).round(2)
print(type_counts)

## Conclusions:
Average and median sizes around 90 pixels

Filling_ratio pretty consistent through classes
- Triangles: 0.5
- Circles: 0.77
- Rectangles: 0.99

# Item 2 - Split training dataset

Extract the 30% of the training images of each class to set up a validation dataset

In [ ]:
# Count number of signals per class
n_signals = df['type'].value_counts(sort=False).reindex(['A','B','C','D','E','F'])
print(n_signals)

# Plot
df['type'].value_counts(sort=False).reindex(['A','B','C','D','E','F']).plot(figsize=(10, 7),kind='bar',sort_columns=True)


In [ ]:
# Get only one class
df_filtered = df[df['type'] == "B"]
df_filtered

In [ ]:
df_sorted_train = df_filtered.sample(frac=0.7)
df_sorted_train

In [ ]:
# Save train selection
tuple(zip(df_sorted_train.index.get_level_values(0).tolist(),df_sorted_train.index.get_level_values(1).tolist()))


In [ ]:
# Delete train selection to get validation selection
df_sorted_test = pd.concat([df_filtered,df_sorted_train]).drop_duplicates(keep=False)
tuple(zip(df_sorted_test.index.get_level_values(0).tolist(),df_sorted_test.index.get_level_values(1).tolist()))

Functions to do all the process in one cell

In [ ]:
def split_class(signal_class, train_percentage):
    
    # Choose one signal class
    df_filtered = df[df['type'] == signal_class]
    
    # Sample randomly the percentage choosen
    df_sorted_train = df_filtered.sample(frac=train_percentage)
    
    # Save train selection
    train_images = tuple(zip(df_sorted_train.index.get_level_values(0).tolist(),df_sorted_train.index.get_level_values(1).tolist()))
    
    # Delete train selection to get validation selection
    df_sorted_test = pd.concat([df_filtered,df_sorted_train]).drop_duplicates(keep=False)
    val_images = tuple(zip(df_sorted_test.index.get_level_values(0).tolist(),df_sorted_test.index.get_level_values(1).tolist()))
    
    return train_images, val_images

def split_dataset(data,percentage, classes):
    
    train_images = []
    val_images = []
    
    for signal_class in classes:
        temp_train_images, temp_val_images = split_class(signal_class, percentage)
        train_images += temp_train_images
        val_images += temp_val_images
    return train_images, val_images


classes_red = ['A','B','C']
classes_blue = ['D','F']
classes_mix = ['E']

train_images_red, val_images_red = split_dataset(data, 0.7, classes_red)
train_images_blue, val_images_blue = split_dataset(data, 0.7, classes_blue)
train_images_mix, val_images_mix = split_dataset(data, 0.7, classes_mix)

train_images = train_images_red + train_images_blue + train_images_mix
val_images = val_images_red + val_images_blue + val_images_mix

#print(train_images)
#print(val_images)
    

# Item 3 - Separation by colour


Para cada imagen de prueba, nos basamos en el diccionario y usamos el bounding box que ya calculamos.

Recortamos la imagen, le calculamos el histograma. Sumamos los histogramas de todas las imágenes y obtenemos la suma total.

Luego hacemos lo mismo y al recortar la roi de cada imagen convertimos la roi de rgb a hsv, calculamos el histograma
y obtenemos la suma de todos los histogramas. 

Con estas cosas deberíamos poder obtener los thresholds a aplicar luego.


Hacer gráficos chetos de los histogramas. Ajustar con multiples gaussianas, calcular promedio, mediana, std, etc.
Compararlos y elegir el mejor.

Luego aplicar las máscaras con los thresholds calculados

In [ ]:
# Convert RGB images tu HUE images

# Get stats from HUE images

# Plot and compare

# Select thresholds (RGB y HUE)

# Create masks using previous thresholds



# Calculate histograms:

In [ ]:
from skimage import color
np.seterr(divide='ignore', invalid='ignore')

path_jpg = os.path.join('dataset', 'train')    
dirs_jpg = os.listdir(path_jpg)

##############################
# Calculate color histograms:
##############################

def color_histogram(gt_dictionary, train_selec, path_jpg, color_space):

    c0_hist = np.zeros(255)
    c1_hist = np.zeros(255)
    c2_hist = np.zeros(255)

    for gt, position in train_selec:
        v = gt_dictionary[gt][position]

        jpg_name = gt.replace('gt.', '').replace('txt', 'jpg')
        mask_name = gt.replace('gt', 'mask').replace('txt', 'png')

        jpg_roi = imageio.imread(os.path.join(path_jpg, jpg_name))[v['tly']:v['bry'], v['tlx']:v['brx']]
        mask_roi = imageio.imread(os.path.join(path_mask, mask_name))[v['tly']:v['bry'], v['tlx']:v['brx']]

        if color_space=='rgb':
            final_roi = jpg_roi
            r0 = 0
            rf = 255

        elif color_space=='hsv':
            final_roi = color.rgb2hsv(jpg_roi) 
            r0 = 0
            rf = 1

        elif color_space=='ycbcr':
            final_roi = color.rgb2ycbcr(jpg_roi)
            r0 = 0
            rf = 255

        elif color_space=='xyz':
            final_roi = color.rgb2xyz(jpg_roi)
            r0 = 0
            rf = 1

        mask_roi[mask_roi==0] = 0
        mask_roi[mask_roi!=0] = 1

        bins = np.histogram(final_roi[:,:,0] * mask_roi, bins=255, range=(r0,rf))[1]
        c0_hist += np.histogram(final_roi[:,:,0] * mask_roi, bins=255, range=(r0,rf))[0]
        c1_hist += np.histogram(final_roi[:,:,1] * mask_roi, bins=255, range=(r0,rf))[0]
        c2_hist += np.histogram(final_roi[:,:,2] * mask_roi, bins=255, range=(r0,rf))[0]

    return bins, c0_hist, c1_hist, c2_hist, r0, rf


######################################
# Calculate RGB normalized histogram:
######################################

def norm_histogram(gt_dictionary, train_selec, path_jpg, color_space):

    c0_hist = np.zeros(255)
    c1_hist = np.zeros(255)
    c2_hist = np.zeros(255)
    
    for gt, position in train_selec:
        v = gt_dictionary[gt][position]

        jpg_name = gt.replace('gt.', '').replace('txt', 'jpg')
        mask_name = gt.replace('gt', 'mask').replace('txt', 'png')

        # Important: in order to normalize we need to read THE FULL IMAGE. If we normalize the rois, 
        # we will be training our algorithm poorly.After trying this, we saw all images have saturated
        # pixels (aka normalized image = original image). 

        # We will normalize the ROIs, knowing this is poorly training the algorithm.

        jpg_roi = imageio.imread(os.path.join(path_jpg, jpg_name))[v['tly']:v['bry'], v['tlx']:v['brx']]
        mask_roi = imageio.imread(os.path.join(path_mask, mask_name))[v['tly']:v['bry'], v['tlx']:v['brx']]

        jpg_max_0 = np.max(jpg_roi[:,:,0])
        jpg_max_1 = np.max(jpg_roi[:,:,1])
        jpg_max_2 = np.max(jpg_roi[:,:,2])

        if color_space=='rgb':
            final_roi = jpg_roi
            jpg_max_0, jpg_max_1, jpg_max_2 = jpg_max_0, jpg_max_1, jpg_max_2 

        mask_roi[mask_roi==0] = 0
        mask_roi[mask_roi!=0] = 1

        bins = np.histogram(final_roi[:,:,0] * mask_roi, bins=255, range=(0,1))[1]
        c0_hist += np.histogram(final_roi[:,:,0] / jpg_max_0 * mask_roi, bins=255, range=(0,1))[0]
        c1_hist += np.histogram(final_roi[:,:,1] / jpg_max_1 * mask_roi, bins=255, range=(0,1))[0]
        c2_hist += np.histogram(final_roi[:,:,2] / jpg_max_2 * mask_roi, bins=255, range=(0,1))[0]

    return bins, c0_hist, c1_hist, c2_hist


###################
# Plot histograms:
###################

def plot_histogram(hist0, hist1, hist2, r0, rf, color_name):
    
    path_figures = os.path.join('figures')
    try:
        os.stat(path_figures)
    except:
        os.mkdir(path_figures)

    x = np.linspace(r0, rf, 255)
    
    fig, axs = plt.subplots(1, 3, figsize=(18, 6), sharey=True)
    
    axs[0].bar(x[:-2], hist0[1:-1], color='r', width=0.8*(rf-r0)/255, label='Ch1')
    axs[1].bar(x[:-2], hist1[1:-1], color='g', width=0.8*(rf-r0)/255, label='Ch2')
    axs[2].bar(x[:-2], hist2[1:-1], color='b', width=0.8*(rf-r0)/255, label='Ch3')

    axs[0].legend()
    axs[1].legend()
    axs[2].legend()

    axs[0].set_xlabel('8bit quantification')
    axs[1].set_xlabel('8bit quantification')
    axs[2].set_xlabel('8bit quantification')
    axs[0].set_ylabel('Total number of px')
    
    fig.suptitle(color_name + ' histogram')
    plt.savefig('figures/' + str(color_name) + '_hist.png')
    plt.show()


# Calculate histograms in different color spaces:

In order to decide the color thresholds, we perform a histogram over the signals of interest.
For this purpose, we split the signals by color and calculate the histogram of each group.


RGB histograms:

In [ ]:
bins, r_hist_red, g_hist_red, b_hist_red, rgb0, rgbf = color_histogram(data, train_images_red, path_jpg, 'rgb')
bins, r_hist_blue, g_hist_blue, b_hist_blue, rgb0_blue, rgbf = color_histogram(data, train_images_blue, path_jpg, 'rgb')
bins, r_hist_mix, g_hist_mix, b_hist_mix, rgb0, rgbf = color_histogram(data, train_images_mix, path_jpg, 'rgb')

Normalized RGB histograms:

**Important:** in order to normalize we need to read THE FULL IMAGE. If we normalize the rois, 
we will be training our algorithm poorly.After trying this, we saw all images have saturated
pixels (aka normalized image = original image). 

We will normalize the ROIs, knowing this is poorly training the algorithm.

In [ ]:
bins_norm, r_hist_norm_red, g_hist_norm_red, b_hist_norm_red = norm_histogram(data, train_images_red, path_jpg, 'rgb')
bins_norm, r_hist_norm_blue, g_hist_norm_blue, b_hist_norm_blue = norm_histogram(data, train_images_blue, path_jpg, 'rgb')
bins_norm, r_hist_norm_mix, g_hist_norm_mix, b_hist_norm_mix = norm_histogram(data, train_images_mix, path_jpg, 'rgb')


HSV histograms:


In [ ]:
hbins, h_hist_red, s_hist_red, v_hist_red, hsv0, hsvf = color_histogram(data, train_images_red, path_jpg, 'hsv')
hbins, h_hist_blue, s_hist_blue, v_hist_blue, hsv0, hsvf = color_histogram(data, train_images_blue, path_jpg, 'hsv')
hbins, h_hist_mix, s_hist_mix, v_hist_mix, hsv0, hsvf = color_histogram(data, train_images_mix, path_jpg, 'hsv')

yCbCr histograms:

In [ ]:
ybins, y_hist_red, cb_hist_red, cr_hist_red, ycbcr0, ycbcrf = color_histogram(data, train_images_red, path_jpg, 'ycbcr')
ybins, y_hist_blue, cb_hist_blue, cr_hist_blue, ycbcr0, ycbcrf = color_histogram(data, train_images_blue, path_jpg, 'ycbcr')
ybins, y_hist_mix, cb_hist_mix, cr_hist_mix, ycbcr0, ycbcrf = color_histogram(data, train_images_mix, path_jpg, 'ycbcr')

X,Y,Z histograms:

In [ ]:
xbins, xx_hist_red, yy_hist_red, zz_hist_red, xyz0, xyzf = color_histogram(data, train_images_red, path_jpg, 'xyz')
xbins, xx_hist_blue, yy_hist_blue, zz_hist_blue, xyz0, xyzf = color_histogram(data, train_images_blue, path_jpg, 'xyz')
xbins, xx_hist_mix, yy_hist_mix, zz_hist_mix, xyz0, xyzf = color_histogram(data, train_images_mix, path_jpg, 'xyz')

# Plot histograms in different color spaces:

Plot RGB histograms:

In [ ]:
plot_histogram(r_hist_red, g_hist_red, b_hist_red, rgb0, rgbf, 'RGB - Red Signals')
plot_histogram(r_hist_blue, g_hist_blue, b_hist_blue, rgb0, rgbf, 'RGB - Blue Signals')
plot_histogram(r_hist_mix, g_hist_mix, b_hist_mix, rgb0, rgbf, 'RGB - Mixed Signals')

Plot normalized RGB histograms:

In [ ]:
plot_histogram(r_hist_norm_red, g_hist_norm_red, b_hist_norm_red, rgb0, rgbf, 'RGB Norm - Red Signals')
plot_histogram(r_hist_norm_blue, g_hist_norm_blue, b_hist_norm_blue, rgb0, rgbf, 'RGB Norm - Blue Signals')
plot_histogram(r_hist_norm_mix, g_hist_norm_mix, b_hist_norm_mix, rgb0, rgbf, 'RGB Norm - Mixed Signals')

Plot HSV histograms:

In [ ]:
plot_histogram(h_hist_red, s_hist_red, v_hist_red, hsv0, hsvf, 'HSV - Red signals')
plot_histogram(h_hist_blue, s_hist_blue, v_hist_blue, hsv0, hsvf, 'HSV - Blue signals')
plot_histogram(h_hist_mix, s_hist_mix, v_hist_mix, hsv0, hsvf, 'HSV - Mixed signals')

Plot yCbCr histograms:

In [ ]:
plot_histogram(y_hist_red, cb_hist_red, cr_hist_red, ycbcr0, ycbcrf, 'yCbCr - Red signals')
plot_histogram(y_hist_blue, cb_hist_blue, cr_hist_blue, ycbcr0, ycbcrf, 'yCbCr - Blue signals')
plot_histogram(y_hist_mix, cb_hist_mix, cr_hist_mix, ycbcr0, ycbcrf, 'yCbCr - Mixed signals')

Plot xyz histograms:

In [ ]:
plot_histogram(xx_hist_red, yy_hist_red, zz_hist_red, xyz0, xyzf, 'XYZ - Red signals')
plot_histogram(xx_hist_blue, yy_hist_blue, zz_hist_blue, xyz0, xyzf, 'XYZ - Blue signals')
plot_histogram(xx_hist_mix, yy_hist_mix, zz_hist_mix, xyz0, xyzf, 'XYZ - Mixed signals')

# Normalized RGB histogram:

In [ ]:
from skimage import color

path_jpg = os.path.join('dataset', 'train')
dirs_jpg = os.listdir(path_jpg)

def norm_histogram(gt_dictionary, train_selec, path_jpg, color_space):

    c0_hist = np.zeros(255)
    c1_hist = np.zeros(255)
    c2_hist = np.zeros(255)
    
    for gt, position in train_selec:
        v = gt_dictionary[gt][position]

        jpg_name = gt.replace('gt.', '').replace('txt', 'jpg')
        mask_name = gt.replace('gt', 'mask').replace('txt', 'png')

        # Important: in order to normalize we need to read THE FULL IMAGE. If we normalize the rois, 
        # we will be training our algorithm poorly.After trying this, we saw all images have saturated
        # pixels (aka normalized image = original image). 

        # We will normalize the ROIs, knowing this is poorly training the algorithm.

        jpg_roi = imageio.imread(os.path.join(path_jpg, jpg_name))[v['tly']:v['bry'], v['tlx']:v['brx']]
        mask_roi = imageio.imread(os.path.join(path_mask, mask_name))[v['tly']:v['bry'], v['tlx']:v['brx']]

        jpg_max_0 = np.max(jpg_roi[:,:,0])
        jpg_max_1 = np.max(jpg_roi[:,:,1])
        jpg_max_2 = np.max(jpg_roi[:,:,2])

        if color_space=='rgb':
            final_roi = jpg_roi
            jpg_max_0, jpg_max_1, jpg_max_2 = jpg_max_0, jpg_max_1, jpg_max_2 

        mask_roi[mask_roi==0] = 0
        mask_roi[mask_roi!=0] = 1

        bins = np.histogram(final_roi[:,:,0] * mask_roi, bins=255, range=(0,1))[1]
        c0_hist += np.histogram(final_roi[:,:,0] / jpg_max_0 * mask_roi, bins=255, range=(0,1))[0]
        c1_hist += np.histogram(final_roi[:,:,1] / jpg_max_1 * mask_roi, bins=255, range=(0,1))[0]
        c2_hist += np.histogram(final_roi[:,:,2] / jpg_max_2 * mask_roi, bins=255, range=(0,1))[0]

    return bins, c0_hist, c1_hist, c2_hist


################################################
# Calculate histograms with traffic signal data:
################################################

# RGB histograms:

bins_norm, r_hist_norm, g_hist_norm, b_hist_norm = norm_histogram(data, train_images, path_jpg, 'rgb')



# Plot normalized histogram:

In [ ]:
plot_histogram(r_hist_norm, 0, 1, 'Red_norm', 'r')
plot_histogram(g_hist_norm, 0, 1, 'Green_norm', 'g')
plot_histogram(b_hist_norm, 0, 1, 'Blue_norm', 'b')

Comparar las máscaras obtenidas con el ground truth.

Podemos ver si sirven las funciones que ya nos dieron hechas.

## Conclusions: 
We only study the colors of the signals, so we expect to have False Positives of the same colors in the background

# Lo que sigue a partir de aqui fueron pruebas hechas el martes:

In [ ]:
print(data['gt.00.005025.txt'][0])
print(data['gt.00.005025.txt'][1])

for gt in dirs_txt:
    print(data[gt][0]['filling_ratio'])

In [ ]:
for mask in dirs_mask:
    gt = mask.replace('mask', 'gt').replace('png', 'txt')
    m = (imageio.imread(os.path.join(path_mask, mask)))
    mask_area = m.sum()
    d = data[gt]
    d['mask_area'] = mask_area
    d['filling_ratio'] = mask_area / d['bbox_area']
    
    print(d['mask_area'], d['filling_ratio'], mask)
    

In [ ]:
print(dirs_mask[0].replace('mask', 'gt').replace('png', 'txt'))
print(dirs_txt[0])


In [ ]:
import matplotlib.pyplot as plt

% matplotlib inline

m = imageio.imread(os.path.join(path_mask,'mask.00.005025.png')).astype(np.int8)

plt.imshow(m[146:201, 1324:1375])
